In [1]:
from sys import platform
import os
import cv2
import json
from pupil_apriltags import Detector
import open3d as o3d
from glob import glob
import numpy as np
from scipy.spatial.transform import Rotation as R
import collections
import k3d
import PoseEstimation.Mesh_Matcher

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
from math import pi
def position_error(T_e, T_gt):

    return np.linalg.norm(T_e- T_gt)


def rotation_error(R_e, R_gt):
    
    return (180/pi) * np.arccos((np.trace(np.matmul(R_e, R_gt.T))-1)/2)

In [3]:
def k3d_frustrum(pose, name, size=0.009, color=0x0000ff):
    # i.e. not using image sizes 
    pos = pose[:3, 3]
    
    forward = -pose[:3, 2] * size * -1.4
    right = -pose[:3, 0] * size * 1.25
    up = -pose[:3, 1] * size
    
    #verts = [pos, pos + forward*size ]
    verts = [pos, pos + forward - right*0.5 + up*0.5, pos + forward + right * 0.5 + up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 - up*0.5, pos + forward + right * 0.5 - up * 0.5, pos ]
    verts += [pos, pos + forward - right*0.5 + up*0.5, pos + forward - right * 0.5 - up * 0.5, pos]
    verts += [pos, pos + forward + right*0.5 + up*0.5, pos + forward + right * 0.5 - up * 0.5, pos]
    return k3d.line(verts, color=color, width=2.5, name = name, shader="simple")

def visualization_registration(reg, mesh):
    vertices = [ ]

    frustrums = []

    colors = [0xff0000, 0x00ff00, 0x0000ff, 0xffcc00, 0xccff00, 0x00ccff]

    for i, frame in enumerate(reg.keys()):
        pos = reg[frame][:3, 3] 
        vertices += pos.tolist()
        frustrums.append( k3d_frustrum(reg[frame], name = str(frame), size=0.1, color=colors[i % len(colors)]) )

    vertices = np.array(vertices)

    lines = k3d.line(vertices, color=0xff0000, width=2.5, shader="simple") # + line
    pts = k3d.points(vertices, point_size=0.003)

    plot3d = k3d.plot(antialias=True, camera_auto_fit=True)

    #plot3d += lines
    plot3d += pts 

    for f in frustrums:
        plot3d += f

    plot3d += k3d.points( [vertices[:3]], point_size=0.0075, color=0x00ff00)

    plot3d += k3d.mesh(np.array(mesh.vertices), np.array(mesh.triangles).flatten(), color=0xffcc00)
    plot3d.display()

at_detector = Detector(
            families="tagStandard41h12",
            nthreads=1,
            quad_decimate=1.0,
            quad_sigma=0.0,
            refine_edges=1,
            decode_sharpening=0.25,
            debug=0,
        )


Register = collections.namedtuple(
    "RegisterInfo", ["CameraPose", "Intrinsic", "TagPose"])


In [4]:
if platform == "linux" or platform == "linux2":  
# linux
    data_path  = "/home/biyang/Documents/3D_Gaze/dataset/3D_scanner_app/Apriltag1_dataset1"
    data_pi_path = "/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/office1/data_1"
    evaluation_json_path = r"/home/biyang/Documents/3D_Gaze/dataset/PupilInvisible/evaluation_apriltag_detection_Iphone.json"
elif platform == "win32":
# Windows...
    data_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\3D_Scanner_App\Apriltag1_dataset1"
    data_pi_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\PupilInvisible\office1\data_1"
    evaluation_json_path = r"D:\Documents\Semester_Project\3D_Gaze\dataset\evaluation_apriltag_detection_Iphone_a1d1.json"


Passed
Passed
Passed
Passed
Passed
Passed


In [6]:
pos_error = []
rot_error = []
for gt_pose, est_pose in zip(GT_CameraPose.values(), IPhone_Registration.values()):
    pos_error.append(position_error(est_pose[:3, 3], gt_pose[:3, 3]))
    rot_error.append(rotation_error(est_pose[:3, :3], gt_pose[:3, :3]))


In [12]:
print(pos_error)

print(sum(pos_error)/len(pos_error))

[0.4083566926725213, 0.4019291218919338, 0.38849037585498103, 0.3179002427648748, 0.17195424301140128, 0.11458310610614114, 0.14301039197887933, 0.022714978658934494, 0.6297980737814189, 0.6552598802543246, 0.7307978960767659, 0.8091163774747812, 0.8676330072610149, 0.8760745025839376, 0.9575329461968345, 0.9874435095169811, 1.0452871353042466, 1.0755181084883743, 1.1233902567194112, 1.1597960564759109, 0.060254736393963046, 0.0716734155875048, 0.12122319913206606, 0.1498385274470514, 0.16208966344672893, 0.18995614177565434, 0.19998547469113256, 0.20044754530162393, 0.23646957514635358, 2.6106487057365686, 2.619049988957643, 2.6365642349417837, 0.7218254377285517, 0.6183800254919476, 0.39066947748013303, 1.9307558123797426, 0.6714375587603891, 1.709616233650727]
0.741775596240085


In [11]:
print(rot_error)

print(sum(rot_error)/len(rot_error))

[7.36927478990779, 7.187212157400475, 7.115298311392832, 6.261001362692682, 3.9361325580072655, 1.9489432525297643, 3.2921470266853063, 0.9753621536041962, 10.96642390395527, 10.455888527461664, 8.952275416433888, 7.628594003444916, 7.443266153672563, 8.12870438897756, 7.440157897565868, 7.729102133388627, 8.647531289409653, 7.000740773123268, 8.05635237001087, 9.879733784407085, 1.3850683352572002, 3.1138184356959884, 4.581211794640418, 4.727277382194506, 4.557758585460071, 5.460402817512808, 6.205918044248811, 6.7300659156342375, 7.04010462180643, 90.00804364895187, 90.03090195805373, 90.07239123681987, 11.47318425571363, 11.34330534303244, 11.385697549881717, 13.840148057493119, 7.4777019939761376, 19.630427428189975]
13.933620254174592


In [18]:
mesh_textured = o3d.io.read_triangle_mesh(mesh_fullpath)

In [15]:
# Visualization of Camera Pose
#visualization_registration(PI_registration, mesh_textured)
visualization_registration(GT_CameraPose, mesh_textured)

d:\Users\ybMas\anaconda3\envs\apriltag\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
d:\Users\ybMas\anaconda3\envs\apriltag\lib\site-packages\traittypes\traittypes.py:101: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [20]:
pcd_sample = mesh_textured.sample_points_uniformly(number_of_points=10000)
o3d.visualization.draw_geometries([pcd_sample])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Das Handle ist ungültig. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Der angeforderte Transformationsvorgang wird nicht unterstützt. 
